In [25]:
import json
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

BASE_DIR = '../'


In [26]:
import os
import fiona
BASE_DIR = os.path.dirname(os.getcwd())
os.chdir(BASE_DIR + '/src/data/')
import util
os.chdir(BASE_DIR + '/notebooks/')

In [27]:
def is_inter(id):
    if len(str(id)) > 1 and str(id)[0:2] == '00':
        return False
    return True

In [28]:
def summary_crash_rate(crashes, inter_count, non_inter_count):
    """                                                                                                                                                                 
    Info about intersections vs non intersections and their crash rate                                                                                                  
    """
    counts = {
        'inter': 0,
        'non_inter': 0,
        'no_match': 0,
        'inter_plus': 0,
        'non_inter_plus': 0,
    }
    total_crashes = 0
    for k, v in crashes.iteritems():
        total_crashes += v['count']
        if str(k) == '':
            # Sometimes crashes can't be snapped to a segment, probably due to bad data entry from the crash
            counts['no_match'] += 1
        elif is_inter(k):
            if int(v['count']) > 1:
                counts['inter_plus'] += 1
            counts['inter'] += 1
        else:
            if int(v['count']) > 1:
                counts['non_inter_plus'] += 1
            counts['non_inter'] += 1
    print "Total number of crashes:\t\t\t\t" + str(total_crashes)
    print "Total number of crash locations:\t\t\t\t" + str(len(crashes))
    print "Number of intersections:\t\t\t\t" + str(inter_count)
    print "Number of non-intersections:\t\t\t\t" + str(non_inter_count)
    print "Percentage of segments that are intersections:\t\t" \
        + str(round(100 * float(inter_count)/float(inter_count + non_inter_count)))
    print ""

    print "Number of intersections with at least 1 crash:\t\t" + str(counts['inter']) 
    print 'Number of intersections with more than 1 crash:\t\t' + str(counts['inter_plus'])
    print "Number of non-intersections with at least 1 crash:\t" + str(counts['non_inter'])
    print "Number of non-intersections with more than 1 crash:\t" + str(counts['non_inter_plus'])
    print ""

    total_percent = round(float(100 * (counts['inter'] + counts['non_inter']) / float(inter_count + non_inter_count)))
    print "percent of all segments with crash:\t\t\t" + str(total_percent)

    # Percentage of intersections/non-intersections                                                                                                                     
    # that have at least one crash
    inter_percent = round(float(100 * counts['inter'])/float(inter_count))
    print "percent of intersections with crash:\t\t\t" + str(inter_percent)

    non_inter_percent = round(100 * float(counts['non_inter'])/float(non_inter_count))
    print "percentage of non-intersections with crash:\t\t" + str(non_inter_percent)

    print "percentage of intersections with more than 1 crash:\t" + str(
        round(float(100 * counts['inter_plus'])/float(inter_count)))

    print "percentage of non-intersections with more than 1 crash:\t" + str(
        round(float(100 * counts['non_inter_plus'])/float(non_inter_count)))


In [29]:
def summary_concern_counts(crashes, concerns):
    """                                                                                                                                                                 
    What percentage of intersections with concerns had crashes                                                                                                          
    at varying counts of concerns?                                                                                                                                      
    """

    matching = {}
    # Go through each concern location
    # Increment counts for crash/no crash intersection/no intersection at this location
    for id, d in concerns.iteritems():
        if d['count'] not in matching.keys():
            matching[d['count']] = {
                'inter': {'crash': 0, 'no_crash': 0},
                'non_inter': {'crash': 0, 'no_crash': 0}
            }
        if is_inter(id):
            key = 'inter'
        else:
            key = 'non_inter'

        if id in crashes.keys():
            matching[d['count']][key]['crash'] += 1
        else:
            matching[d['count']][key]['no_crash'] += 1

    sorted_matching = sorted(matching.items())

    results = []
    for key, value in sorted_matching:

        # Do the 1+,2+ stats as well as 1, 2                                                                                                                            
        # Still need to break it out by int/non-int                                                                                                                     
        counts = {
            'total': value['inter']['crash'] + value['inter']['no_crash'] \
                + value['non_inter']['crash'] + value['non_inter']['no_crash'],
            'crashes': value['inter']['crash'] + value['non_inter']['crash'],
            'inters_total': value['inter']['crash'] + value['inter']['no_crash'], # Count of intersections with a concern
            'inters_crashes': value['inter']['crash'],
            'non_inters_total': value['non_inter']['crash'] + value['non_inter']['no_crash'],
            'non_inters_crashes':  value['non_inter']['crash'],
        }
        # Add all the data for segments with more complaints than the current complaint # we're on
        for key2, value2 in sorted_matching[key:len(sorted_matching)]:

            if key2 > key:
                counts['total'] += value2['inter']['crash'] + value2['inter']['no_crash'] \
                    + value2['non_inter']['crash'] + value2['non_inter']['no_crash']
                counts['crashes'] += value2['inter']['crash'] + value2['non_inter']['crash']
                counts['inters_total'] += value2['inter']['crash'] + value2['inter']['no_crash']
                counts['inters_crashes'] += value2['inter']['crash']
                counts['non_inters_total'] += value2['non_inter']['crash'] + value2['non_inter']['no_crash']
                counts['non_inters_crashes'] += value2['non_inter']['crash']
        total_percent_v0 = round(100 * float(counts['crashes'])/float(counts['total']))
        inter_percent_v0 = round(100 * float(counts['inters_crashes'])/float(counts['inters_total'])) if counts['inters_total'] else 0
        non_inter_percent_v0 = round((100 * float(counts['non_inters_crashes'])/float(counts['non_inters_total'])) if counts['non_inters_total'] else 0)
        results.append([
            key,
            total_percent_v0,
            counts['total'], # total count      
            inter_percent_v0,
            counts['inters_total'], # total # of intersections with this many or more complaints
            non_inter_percent_v0,
            counts['non_inters_total'], # total # of non-intersections with this many or more complaints
            round(100 * float(counts['inters_total'])/float(counts['total']))
        ])

    results = results[0:5]

    display(pd.DataFrame(results, columns=[
        '# of concerns at this segment',
        '% of segments w/ v0 complaint w/ crash',
        'total # of segments with this many or more concerns',
        '% of intersections with this many or more concerns with a crash',
        '# of intersections with this many concerns',
        '% of non-intersections with this many concerns with a crash',
        '# of non-intersections with this many concerns',
        '% of these locations that were at an intersection'
    ]))

In [30]:
def concerns_by_type(concerns, concern_data, crashes, category_field='REQUEST_TYPE'):
    # For intersections with a specific complaint type:                                                                                                                 
    #    what percentage had a crash?                                                                                                                                   
    requests = {}
    for data in concern_data:
        if data[category_field] not in requests.keys():
            requests[data[category_field]] = 1

    requests = {}
    all_unique = []

    for k, v in concerns.iteritems():
        unique_requests = {}
        for request in v[category_field]:
            # Clean up badly formatted request types                                                                                                                    
            vals = request.split('nbsp;')
            if len(vals) > 1:
                request = vals[1]

            if request not in unique_requests.keys():
                unique_requests[request] = 0
            unique_requests[request] += 1

            if request not in requests.keys():
                requests[request] = {
                    'crashes': 0,
                    'total': 0,
                    'inter_crashes': 0,
                    'inter_total': 0,
                    'non_inter_crashes': 0,
                    'non_inter_total': 0,
                    'ped_inter_crashes': 0,
                    'ped_non_inter_crashes': 0
                }
            if str(k) in crashes.keys():
                requests[request]['crashes'] += 1
            requests[request]['total'] += 1
            if is_inter(k):
                if str(k) in crashes.keys():
                    requests[request]['inter_crashes'] += 1
                requests[request]['inter_total'] += 1
            else:
                if str(k) in crashes.keys():
                    requests[request]['non_inter_crashes'] += 1
                requests[request]['non_inter_total'] += 1


        for key, value in unique_requests.iteritems():
            if value > 1:
                all_unique.append([key, value])

    by_type = {}
    for k, v in all_unique:
        if k not in by_type.keys():
            by_type[k] = 0
        by_type[k] += 1

    results = []
    for k, v in requests.iteritems():
        if requests[k]['total'] >= 100:
            total_percent = round(100 * float(requests[k]['crashes'])/float(requests[k]['total']))
            inter_percent = round(100 * float(requests[k]['inter_crashes'])/float(requests[k]['inter_total']))
            non_inter_percent = round(100 * float(requests[k]['non_inter_crashes'])/float(requests[k]['non_inter_total']) if requests[k]['non_inter_total'] else 0)
            results.append([
                k,
                by_type[k],
                total_percent,
                requests[k]['crashes'],
                requests[k]['total'],
                inter_percent,
                requests[k]['inter_crashes'],
                requests[k]['inter_total'],
                non_inter_percent,
                requests[k]['non_inter_crashes'],
                requests[k]['non_inter_total'],
                round(100 * float(requests[k]['inter_total'])/float(requests[k]['total']))
            ])


    df = pd.DataFrame(results, columns=[
        'Request type',
        '# of requests of this type that appear more than once at a segment',
        '% of segments with this type of request with a crash',
        '# of segments with this type of request with a crash',
        '# of segments with this type of a request total',
        '% of intersections with this type of request with a crash',
        '# of intersections with this type of request with a crash',
        '# of intersections with this type of request total',
        '% of non-intersections with this type of request with a crash',
        '# of non-intersections with this type of request with a crash',
        '# of non-intersections with this type of request total',
        '% of complaint locations that were intersections'
        #'% of intersections with this type of request with a ped crash',
        #'# of ped crashes in intersection with this request',
        #'# of ped crashes in non-intersection with this request'
    ])
    df.sort_values(by="# of segments with this type of a request total", ascending=0)
    display(df)


In [31]:
def get_analysis_for_city(datadir, category_field='category', years=None):
    
    DATA_FP = BASE_DIR + '/' + datadir + '/processed/'
    MAP_FP = DATA_FP + 'maps/'
    items = json.load(open(DATA_FP + 'crash_joined.json'))
    crash_data, crashes = util.group_json_by_location(items, years=years, yearfield='dateOccurred')

    items = json.load(open(DATA_FP + 'concern_joined.json'))
    concern_data, concerns = util.group_json_by_location(
        items,
        otherfields=[category_field])
    print 'len concern data:' + str(len(concern_data))
    inters = util.read_segments(MAP_FP, get_non_inter=False)
    non_inters = util.read_segments(MAP_FP, get_inter=False)
    inter_count = len(inters[0])
    non_inter_count = len(non_inters[0])
    
    summary_crash_rate(crashes, inter_count, non_inter_count)
    summary_concern_counts(crashes, concerns)
    concerns_by_type(concerns, concern_data, crashes, category_field)
    

In [32]:
get_analysis_for_city('osm-data', years=[2016])

len concern data:13369
Read in 7845 intersection, 0 non-intersection segments
Read in 0 intersection, 15434 non-intersection segments
Total number of crashes:				3202
Total number of crash locations:				1958
Number of intersections:				7845
Number of non-intersections:				15434
Percentage of segments that are intersections:		34.0

Number of intersections with at least 1 crash:		1110
Number of intersections with more than 1 crash:		397
Number of non-intersections with at least 1 crash:	847
Number of non-intersections with more than 1 crash:	152

percent of all segments with crash:			8.0
percent of intersections with crash:			14.0
percentage of non-intersections with crash:		5.0
percentage of intersections with more than 1 crash:	5.0
percentage of non-intersections with more than 1 crash:	1.0


,# of concerns at this segment,% of segments w/ v0 complaint w/ crash,total # of segments with this many or more concerns,% of intersections with this many or more concerns with a crash,# of intersections with this many concerns,% of non-intersections with this many concerns with a crash,# of non-intersections with this many concerns,% of these locations that were at an intersection
0,1,17.0,5954,27.0,2361,10.0,3593,40.0
1,2,24.0,2546,37.0,1093,14.0,1453,43.0
2,3,29.0,1368,45.0,614,16.0,754,45.0
3,4,33.0,840,49.0,398,19.0,442,47.0
4,5,37.0,569,53.0,277,21.0,292,49.0


,Request type,# of requests of this type that appear more than once at a segment,% of segments with this type of request with a crash,# of segments with this type of request with a crash,# of segments with this type of a request total,% of intersections with this type of request with a crash,# of intersections with this type of request with a crash,# of intersections with this type of request total,% of non-intersections with this type of request with a crash,# of non-intersections with this type of request with a crash,# of non-intersections with this type of request total,% of complaint locations that were intersections
0,None,2546,28.0,3725,13369,43.0,2516,5900,16.0,1209,7469,44.0


In [33]:
get_analysis_for_city('osm-dc', years=[2016])

KeyError: 'dateOccurred'

In [ ]:
get_analysis_for_city('osm-cambridge', category_field='issue_type', years=[2016])

In [ ]:
# Why are there things like turning complaints at non-intersections?
# Pedestrian vs. bike vs. car
# Compare counts against ATR/TMC volume counts
# Parsing extra field's text?